# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [77]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

Archivo esta comprimido, le ponemos el parametro al read_csv

In [78]:
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')
#print and see:
#loans


/tmp/ipykernel_42550/4019059519.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip')


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [79]:
column_types =loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

Tuve que instalar openpyxl.

In [80]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [81]:
#datos_dict

Reordenar para que esten en mismo orden que el df

In [82]:
# Assuming 'loans' is your DataFrame
loans_columns = loans.columns.tolist()  # Get the list of columns from the loans DataFrame

# Reorder datos_dict based on the columns in loans
# We will filter out features that are not in loans_columns
ordered_dict = datos_dict[datos_dict['feature'].isin(loans_columns)].copy()
ordered_dict = ordered_dict.set_index('feature').reindex(loans_columns).reset_index()

# Display the ordered table
#print(ordered_dict)


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [83]:
# COdigo guardar
datos_dict.to_pickle('ordered_dict.pkl')


In [84]:
# Codigo para cargar
datos_dict_loaded = pd.read_pickle('ordered_dict.pkl')


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

Unnamed0, esta columna es identica al indice del dataframe, asi que la vamos a eliminar.

In [85]:
# Manejos de Unnamed 0
loans = loans.drop(columns=['Unnamed: 0'])


El term lo cambiaremos a numero, quitamos months y solo dejamos 36 o 60

In [86]:
#term
# Get unique values from a specific column (e.g., 'id')
unique_ids = loans['term'].unique()

# Print the unique values
#print(unique_ids)

# Assuming your DataFrame is named 'loans' and the column is named 'term', extraemos solo numero y casteamos a int.
loans['term'] = loans['term'].str.extract('(\d+)').astype(int)



<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_42550/4179978830.py:9: SyntaxWarning: invalid escape sequence '\d'
  loans['term'] = loans['term'].str.extract('(\d+)').astype(int)


Cambiamos grades y sub_grades a category, ya que son categorical data.

In [87]:
#grades y subgrades
sorted_unique_grades = sorted(loans['grade'].unique())
sorted_unique_sub_grades = sorted(loans['sub_grade'].unique())

#print("Sorted Unique grades:", sorted_unique_grades)
#print("Sorted Unique sub_grades:", sorted_unique_sub_grades)

# Cast the columns to 'category' dtype
loans['grade'] = loans['grade'].astype('category')
loans['sub_grade'] = loans['sub_grade'].astype('category')

emp_title a string, 

In [88]:
#emp_title
# Get unique values from the 'emp_title' column
unique_titles = loans['emp_title'].unique()

# Convert to list and print
unique_titles_list = unique_titles.tolist()
#print(unique_titles_list)

loans['emp_title'] = loans['emp_title'].astype(str)



emp_length
Notar la descripcion                                   emp_length                                                                                                         
Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
Asi que cambiaremos a numerico como lo indica la descripcion

In [89]:
unique_titles = loans['emp_length'].unique()

# Convert to list and print
unique_titles_list = unique_titles.tolist()
#print(unique_titles_list)

# Create a mapping dictionary for the replacements
length_mapping = {
    '10+ years': 10,
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9
}

# Replace the values in the emp_length column using the mapping
loans['emp_length'] = loans['emp_length'].replace(length_mapping)

# Convert to integer type, preserving NaNs
loans['emp_length'] = loans['emp_length'].astype('Int64')  # 'Int64' allows for NaN values


/tmp/ipykernel_42550/228502096.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  loans['emp_length'] = loans['emp_length'].replace(length_mapping)


home_ownership
Descripcion dice Our values are: RENT, OWN, MORTGAGE, OTHER
Cambiamos any a other
Convertimos a categorial data

In [90]:
unique_titles = loans['home_ownership'].unique()

# Convert to list and print
unique_titles_list = unique_titles.tolist()
#print(unique_titles_list)

# Replace 'ANY' with 'OTHER' in the 'home_ownership' column
loans['home_ownership'] = loans['home_ownership'].replace('ANY', 'OTHER')

# Convert the 'home_ownership' column to categorical type
loans['home_ownership'] = loans['home_ownership'].astype('category')



Verification status, cambiamos a category

In [91]:
# Check unique values in the 'verification_status' column
unique_verification_status = loans['verification_status'].unique()
#print("Unique values in 'verification_status':", unique_verification_status)

# Convert the 'verification_status' column to categorical type
loans['verification_status'] = loans['verification_status'].astype('category')




issue_d, cambiamos a timestamp

In [92]:
# Assuming loans is your DataFrame
loans['issue_d'] = pd.to_datetime(loans['issue_d'], format='%b-%Y')


loan_status

In [93]:
unique_loan_status = loans['loan_status'].unique()
#print(unique_loan_status)

loans['loan_status'] = loans['loan_status'].astype('category')


pymnt_plan solo tiene y, n; lo haremos boolean

In [94]:
unique_loan_status = loans['pymnt_plan'].unique()
print(unique_loan_status)

loans['pymnt_plan'] = loans['pymnt_plan'].map({'n': False, 'y': True})



['n' 'y']


Desc,  title son descripciones. zip, purpose y addr categorias; purpose (A category provided by the borrower for the loan request.),

In [95]:
# Change data types
loans['desc'] = loans['desc'].astype(str)           # Convert 'desc' to string
loans['purpose'] = loans['purpose'].astype('category')     # Convert 'purpose' to string
loans['title'] = loans['title'].astype(str)         # Convert 'title' to string
loans['zip_code'] = loans['zip_code'].astype('category')  # Convert 'zip_code' to categorical
loans['addr_state'] = loans['addr_state'].astype('category')  # Convert 'addr_state' to categorical



earliest_cr_line, son puras fechas, lo haremos timestamp

In [96]:
loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'], format='%b-%Y', errors='coerce')


initial list status, valores solo w o f, a category, boolean no pq no se si w  significa True y f no es False

In [97]:
loans['initial_list_status'] = loans['initial_list_status'].astype('category')

Son fechas, de object a timestamp

In [98]:
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'], format='%b-%Y', errors='coerce')
loans['last_credit_pull_d'] = pd.to_datetime(loans['last_credit_pull_d'], format='%b-%Y', errors='coerce')
loans['next_pymnt_d'] = pd.to_datetime(loans['next_pymnt_d'], format='%b-%Y', errors='coerce')



policy code solo puede tener valores 1 y 2, haremos category
Application type igual solo puede tener 2 valores, individual o conjunta

In [99]:
loans['policy_code'] = loans['policy_code'].astype('category')
loans['application_type'] = loans['application_type'].astype('category')


hardship, flag to boolean.
type y status a category.
Reason quedara object

In [100]:
# Convert hardship_flag to boolean
loans['hardship_flag'] = loans['hardship_flag'].map({'N': False, 'Y': True})
loans['hardship_status'] = loans['hardship_status'].astype('category')
loans['hardship_type'] = loans['hardship_type'].astype('category')



A timestamp, son fechas

In [101]:
loans['hardship_start_date'] = pd.to_datetime(loans['hardship_start_date'], format='%b-%Y', errors='coerce')
loans['hardship_end_date'] = pd.to_datetime(loans['hardship_end_date'], format='%b-%Y', errors='coerce')
loans['payment_plan_start_date'] = pd.to_datetime(loans['payment_plan_start_date'], format='%b-%Y', errors='coerce')

Viendo los valores unicos y descripciones cambiamos object a su correspondiente en:

In [102]:
# Convert disbursement_method to category
loans['disbursement_method'] = loans['disbursement_method'].astype('category')

# Convert debt_settlement_flag to boolean
loans['debt_settlement_flag'] = loans['debt_settlement_flag'].map({'Y': True, 'N': False})

# Convert debt_settlement_flag_date to timestamp
loans['debt_settlement_flag_date'] = pd.to_datetime(loans['debt_settlement_flag_date'], format='%b-%Y', errors='coerce')

# Convert settlement_status to category
loans['settlement_status'] = loans['settlement_status'].astype('category')

# Convert settlement_date to timestamp
loans['settlement_date'] = pd.to_datetime(loans['settlement_date'], format='%b-%Y', errors='coerce')

In [103]:
column_types =loans.dtypes


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [104]:
#Checamos todas las columnas con missings or nans y sus descripciones para decidir que hacer con esos missings or Nans
# Iterar sobre cada columna y verificar si tiene valores faltantes
'''
columnas_con_faltantes = [col for col in loans.columns if loans[col].isna().any()]

# Mostrar el resultado
if columnas_con_faltantes:
    print("Las columnas con valores faltantes son:", columnas_con_faltantes)
else:
    print("No hay columnas con valores faltantes.")
    
# Print unique values for each column
for column in columnas_con_faltantes:
    if column in loans.columns:  # Check if the column exists in the DataFrame
        unique_values = loans[column].unique()
        print(f"Unique values in '{column}': {unique_values}")
    else:
        print(f"Column '{column}' does not exist in the DataFrame.")
# Column to check
if columnas_con_faltantes:
    X = columnas_con_faltantes[0]

    # Get the description from ordered_dict
    description = ordered_dict[ordered_dict['feature'] == X]['description'].values[0]

    # Get the index position from loans DataFrame columns
    try:
        column_index = loans.columns.get_loc(X)  # Get index position directly from loans columns
        print(f"Column Index: {column_index}")
    except KeyError:
        print(f"Column '{X}' does not exist in the DataFrame.")

    # Print description
    print(f"Description: {description}")

    # Print unique values from the specified column in loans DataFrame
    if X in loans.columns:  # Check if the column exists before accessing it
        unique_values = loans[X].unique()
        print(f"Unique Values in '{X}': {unique_values}")
    else:
        print(f"Column '{X}' does not exist in the DataFrame.")
        
        
    column_types[X]
'''


'\ncolumnas_con_faltantes = [col for col in loans.columns if loans[col].isna().any()]\n\n# Mostrar el resultado\nif columnas_con_faltantes:\n    print("Las columnas con valores faltantes son:", columnas_con_faltantes)\nelse:\n    print("No hay columnas con valores faltantes.")\n    \n# Print unique values for each column\nfor column in columnas_con_faltantes:\n    if column in loans.columns:  # Check if the column exists in the DataFrame\n        unique_values = loans[column].unique()\n        print(f"Unique values in \'{column}\': {unique_values}")\n    else:\n        print(f"Column \'{column}\' does not exist in the DataFrame.")\n# Column to check\nif columnas_con_faltantes:\n    X = columnas_con_faltantes[0]\n\n    # Get the description from ordered_dict\n    description = ordered_dict[ordered_dict[\'feature\'] == X][\'description\'].values[0]\n\n    # Get the index position from loans DataFrame columns\n    try:\n        column_index = loans.columns.get_loc(X)  # Get index position

Todos estos solo tienen valores nans, droppeamos la columna.

In [105]:
c = [column for column in loans.columns if loans[column].isna().all()]

# Iterate over each column and drop it if it exists
for column in c:
    if column in loans.columns:  # Check if the column exists in the DataFrame
        loans.drop(columns=[column], inplace=True)  # Drop the column



Poner 0 puede interpretarse como algo que hizo, debe, falta, o que ha trabajado, cambiaremos a -1 para evitar complicaciones.

In [106]:
# List of columns to fill with -1
amenos1 = [
    'emp_length', 
    'mths_since_last_delinq', 
    'mths_since_last_record', 
    'mths_since_last_major_derog', 
    'mo_sin_old_il_acct', 
    'mths_since_recent_bc', 
    'mths_since_recent_bc_dlq', 
    'mths_since_recent_inq', 
    'mths_since_recent_revol_delinq'
]

# Fill NaN values with -1 for each column in the list
for col in amenos1:
    loans[col] = loans[col].fillna(-1)



revol_util y bc_util, son un porcentaje, cambiare por 0 si su revol_bal es 0, por la mediana si su revol_bal es distinto de 0


In [107]:
# When revolving balance is 0, set utilization to 0
# For other cases, use median imputation
# Define columns and their respective conditions
aMediano0 = ['revol_util', 'bc_util']
median_values = []

for col in aMediano0:
    # Calculate the median for the current column, ignoring NaNs
    median_value = loans[loans[col].notna()][col].median()
    median_values.append(median_value)  # Store median for later use

    # Apply the conditions based on the current column
    if col == 'revol_util':
        loans[col] = loans.apply(
            lambda row: 0 if row['revol_bal'] == 0 
            else median_value if pd.isna(row[col]) 
            else row[col], 
            axis=1
        )
    elif col == 'bc_util':
        loans[col] = loans.apply(
            lambda row: 0 if row['bc_open_to_buy'] == 0 
            else median_value if pd.isna(row[col]) 
            else row[col], 
            axis=1
        )


Para estos de timestamps, los cambiare por una fecha que signifique desconocido

In [108]:
# Example of using a specific placeholder date
aFechaHolder = ['hardship_start_date','last_pymnt_d','next_pymnt_d','last_credit_pull_d','hardship_end_date','payment_plan_start_date',
                'debt_settlement_flag_date','settlement_date']
placeholder_date = pd.Timestamp('1900-01-01')

for i in aFechaHolder:
    loans[i] = loans[i].fillna(placeholder_date)



Cambiar valor a 0 no afecta, terminos donde 0 representa nada y no afecta registro.

In [109]:
aCero = ['bc_open_to_buy', 'num_tl_120dpd_2m', 'percent_bc_gt_75','deferral_term',
         'hardship_amount','hardship_length','hardship_dpd','orig_projected_additional_accrued_interest',
         'hardship_payoff_balance_amount','hardship_last_payment_amount','settlement_amount','settlement_percentage',
         'settlement_term']

for i in aCero:
    loans[i] = loans[i].fillna(0)



Categorias, cambiaremos NANS por categoria MISSING

In [110]:
# List of columns to update
categoriasAMissing = ['hardship_type', 'hardship_reason','hardship_status','hardship_loan_status','settlement_status']

# Iterate over each column and update NaN values to 'MISSING'
for column in categoriasAMissing:
    if column in loans.columns:
        # Check if the column is categorical
        if pd.api.types.is_categorical_dtype(loans[column]):
            # Add 'MISSING' as a category if not already present
            if 'MISSING' not in loans[column].cat.categories:
                loans[column] = loans[column].cat.add_categories(['MISSING'])
        
        # Replace NaN values with 'MISSING'
        loans[column] = loans[column].fillna('MISSING')



/tmp/ipykernel_42550/1800534246.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(loans[column]):


### Codigo para salvar y cargar JSONs

In [111]:
import json

In [112]:
def create_imputation_json():
    # Prepare the imputation information for JSON
    imputation_info = {}

    # Document imputation for dropped columns
    for column in c:
        imputation_info[column] = {
            'estrategia': 'Dropeamos columna - todos valores NaN',
            'valor': None  # No value since the column is dropped
        }
        
    # Fill imputation info for 'ameno1'
    for col in amenos1:
        imputation_info[col] = {
            'estrategia': 'Indicar que no con identificador negativo, un 0 implicaria cosas',
            'valor': -1
        }

    # Fill imputation info for 'aCero'
    for col in aCero:
        imputation_info[col] = {
            'estrategia': 'Identificador para decir que es nada',
            'valor': 0
        }

    # Fill imputation info for 'placeholder_dates'
    for col in aFechaHolder:
        imputation_info[col] = {
            'estrategia': 'Indicador que no hay fecha registrada',
            'valor': '1900-01-01'
        }

    # Fill imputation info for 'revol_util' and 'bc_util'
    for i, col in enumerate(aMediano0):
        # Crear una lista para almacenar las estrategias
        strategies = []
        
        # Estrategia: imputación por mediana
        strategies.append({
            'estrategia': 'median imputation',
            'valor': median_values[i]  # Usar la mediana almacenada
        })
        
        # Estrategia: reemplazar 0 con 0
        strategies.append({
            'estrategia': 'identificador',
            'valor': 0
        })

        # Asignar la lista de estrategias a la clave correspondiente
        imputation_info[col] = strategies

    # Fill imputation info for 'categorical_missing'
    for col in categoriasAMissing:
        imputation_info[col] = {
            'estrategia': 'Identificador categorico',
            'valor': 'MISSING'
        }
        
    all_columns = loans.columns.tolist()#Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.
    for col in all_columns:
        if col not in imputation_info:
            imputation_info[col] = {
                'estrategia': 'no imputation',
                'valor': None  # O el valor que consideres apropiado
            }

    # Save to JSON
    with open('imputation_strategies.json', 'w') as json_file:
        json.dump(imputation_info, json_file, indent=4)


In [113]:
def load_and_print_json(file_path):
    try:
        # Load the JSON file
        with open(file_path, 'r') as json_file:
            imputation_info = json.load(json_file)

        # Print the contents of the JSON file
        print(json.dumps(imputation_info, indent=4))  # Pretty print with indentation
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {file_path}.")


In [114]:
# Example usage
create_imputation_json()
load_and_print_json('imputation_strategies.json')

{
    "member_id": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "annual_inc_joint": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "dti_joint": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "verification_status_joint": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "open_acc_6m": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "open_act_il": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "open_il_12m": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "open_il_24m": {
        "estrategia": "Dropeamos columna - todos valores NaN",
        "valor": null
    },
    "mths_since_rcnt_il": {
        "estrategia": "Dropeamos columna - todos va